This exercise will require you to pull some data from the Qunadl API. Qaundl is currently the most widely used aggregator of financial market data.

As a first step, you will need to register a free account on the http://www.quandl.com website.

After you register, you will be provided with a unique API key, that you should store:

In [1]:
# Store the API key as a string - according to PEP8, constants are always named in all upper case
API_KEY = 'f3Mzez4xp39Y84tX3tYx'

Qaundl has a large number of data sources, but, unfortunately, most of them require a Premium subscription. Still, there are also a good number of free datasets.

For this mini project, we will focus on equities data from the Frankfurt Stock Exhange (FSE), which is available for free. We'll try and analyze the stock prices of a company called Carl Zeiss Meditec, which manufactures tools for eye examinations, as well as medical lasers for laser eye surgery: https://www.zeiss.com/meditec/int/home.html. The company is listed under the stock ticker AFX_X.

You can find the detailed Quandl API instructions here: https://docs.quandl.com/docs/time-series

While there is a dedicated Python package for connecting to the Quandl API, we would prefer that you use the *requests* package, which can be easily downloaded using *pip* or *conda*. You can find the documentation for the package here: http://docs.python-requests.org/en/master/ 

Finally, apart from the *requests* package, you are encouraged to not use any third party Python packages, such as *pandas*, and instead focus on what's available in the Python Standard Library (the *collections* module might come in handy: https://pymotw.com/3/collections/).
Also, since you won't have access to DataFrames, you are encouraged to us Python's native data structures - preferably dictionaries, though some questions can also be answered using lists.
You can read more on these data structures here: https://docs.python.org/3/tutorial/datastructures.html

Keep in mind that the JSON responses you will be getting from the API map almost one-to-one to Python's dictionaries. Unfortunately, they can be very nested, so make sure you read up on indexing dictionaries in the documentation provided above.

In [2]:
# First, import the relevant modules
import requests


In [3]:
# Now, call the Quandl API and pull out a small sample of the data (only one day) to get a glimpse
# into the JSON structure that will be returned
URL_AFX_X = 'https://www.quandl.com/api/v3/datasets/FSE/AFX_X/data.json'

r = requests.get(URL_AFX_X+'?api_key='+API_KEY+'&start_date=2017-01-01&end_date=2017-12-31')

In [5]:
# Inspect the JSON structure of the object you created, and take note of how nested it is,
# as well as the overall structure
data = r.json()
# Data extraction
dataset = data['dataset_data']
TS = dataset['data']

These are your tasks for this mini project:

1. Collect data from the Franfurt Stock Exchange, for the ticker AFX_X, for the whole year 2017 (keep in mind that the date format is YYYY-MM-DD).
2. Convert the returned JSON object into a Python dictionary.
3. Calculate what the highest and lowest opening prices were for the stock in this period.
4. What was the largest change in any one day (based on High and Low price)?
5. What was the largest change between any two days (based on Closing Price)?
6. What was the average daily trading volume during this year?
7. (Optional) What was the median trading volume during this year. (Note: you may need to implement your own function for calculating the median.)

In [8]:
# 3. Calculate what the highest and lowest opening prices were for the stock in this period.
Open_Index = dataset['column_names'].index('Open')
Open_Price = [x[Open_Index] for x in TS]
print(max(x for x in Open_Price if x is not None))
print(min(x for x in Open_Price if x is not None))

53.11
34.0


In [10]:
# 4. What was the largest change in any one day (based on High and Low price)?
High_Index = dataset['column_names'].index('High')
Low_Index = dataset['column_names'].index('Low')
High_Price = [x[High_Index] for x in TS]
Low_Price = [x[Low_Index] for x in TS]
High_Low = [x-y for x in High_Price for y in Low_Price]
print(max(x for x in High_Low if x is not None))

19.92


In [11]:
# 5. What was the largest change between any two days (based on Closing Price)?
Close_Index = dataset['column_names'].index('Close')
Close_Price = [x[Close_Index] for x in TS]
Close_Price_Diff = [Close_Price[i+1]-Close_Price[i] for i in range(len(Close_Price)-1)]
print(max(x for x in Close_Price_Diff if x is not None))

2.559999999999995


In [12]:
# 6. What was the average daily trading volume during this year?
Volume_Index = dataset['column_names'].index('Traded Volume')
Volume = [x[Volume_Index] for x in TS ]
Volume = [x for x in Volume if x is not None]
print(sum(Volume)/len(Volume))

89124.33725490196


In [13]:
# 7. (Optional) What was the median trading volume during this year. (Note: you may need to implement your own function for calculating the median.)
def Cal_Median(x):
    x.sort()
    xlen = len(x)
    if(xlen % 2 == 1):
        return x[int((xlen+1)/2-1)]
    else:
        return (x[int(xlen/2)] + x[int(xlen/2)-1])/2

print(Cal_Median(Volume))

76286.0
